In [1]:
import gen_dataset_810 as mod_810fr
# importer datetime pour typer les donnees de dates des chaines et des videos
import datetime
# module https://810.fr pour traitement des tags des videos
import gen_tags_visu_810 as tv810
# module https://810.fr pour casser les gros dataframe ou activites mensuelles
import gen_activitemensuelle_data_810 as amd810



# Définition des dates limites pour les filtres de https://s01.810.fr
liste_mois = [{"mois" : "Janvier", "dates" : {"datemin" : datetime.datetime(2022, 1, 1, 0, 0, 0), "datemax" : datetime.datetime(2022, 2, 1, 0, 0, 0)}},
                {"mois" : "Février", "dates" : {"datemin" : datetime.datetime(2022, 2, 1, 0, 0, 0), "datemax" : datetime.datetime(2022, 3, 1, 0, 0, 0)}},
                {"mois" : "Mars", "dates" : {"datemin" : datetime.datetime(2022, 3, 1, 0, 0, 0), "datemax" : datetime.datetime(2022, 4, 1, 0, 0, 0)}}]


# Récupération des fichiers générés à partir des API YouTube
folder_candidats = './data_sources/stats_videos/2022_03_26/'
# Création d'un datafram qui agrège les fichiers
dt_candidats = mod_810fr.mb_ds()
# Génération d'un dataframe de vidéos et d'un dataframe de chaîne (exemple dans les blocs qui suivent)
videos_candidats_toutes, chaines_candidats_toutes  = dt_candidats.get_full_data_candidat(folder = folder_candidats)
# Generation d'un dataframe par mois sous la forme d'une liste de [{"mois" : moisassocie, "dataframe" : dataframedumois}]
liste_dtf_mensuel = amd810.un_dtf_par_mois(dtfglobal=videos_candidats_toutes, list_dict_dates=liste_mois)

Imported 11 channel data


In [2]:
candidats = chaines_candidats_toutes.candidat.unique()
candidats

array(['Eric Zemmour', 'Marine Le Pen', 'Anne Hidalgo',
       'Valérie Pécresse', 'Jean Lassalle', 'Jean-Luc Mélenchon',
       'Nathalie Arthaud', 'Nicolas Dupont-Aignan', 'Fabien Roussel',
       'Avec vous', 'Yannick Jadot'], dtype=object)

In [3]:
# videos_candidats_toutes.head(20)

- 1 top tags global, avec la part pour chaque mois
- 1 top tags par mois


- Necessite de parler en nombre de videos Taggees
Qui tags et pourquoi.

In [4]:
datemin = liste_mois[0]["dates"]["datemin"]
datemax = liste_mois[-1]["dates"]["datemax"]

# 1. Eric Zemmour

In [5]:
candidat = "Eric Zemmour"
lst_exceptions = ['zemmour', 'zemmour 2022', 'Z00Z']

In [6]:
# Récupère seulement les vidéos d'un candidat dans le dtf globale sur la base du nom du candidat et des dates limites choisies
videos_can = amd810.get_candidat_subdtf(dtfglobal=videos_candidats_toutes, candidat=candidat, datemin=datemin, datemax=datemax)
# Crée une liste de dataframe par mois
dtf_mensuel_candidat = amd810.un_dtf_par_mois(dtfglobal=videos_can, list_dict_dates=liste_mois)
# Donne les tags les plus utilisés sur la période de temps écoulée entre datemin et datemax
top_tags_global = tv810.liste_top_tags_de_chaines (dtfv = videos_can, top = 15, liste_exceptions= lst_exceptions,graph='non')

In [7]:
dtf_tauxtags_mois = tv810.taux_tags_par_mois(listedtfmois=dtf_mensuel_candidat)
dtf_tauxtags_mois.head(20)

,Janvier,Février,Mars
avec tags,100,100,100
sans tags,0,0,0


In [8]:
videos_can = videos_candidats_toutes[videos_candidats_toutes['candidat'] == candidat]
videos_can = videos_can[videos_can['dateVideo'] > datetime.datetime(2022,1,1,0,0,0)]
videos_can = videos_can[videos_can['dateVideo'] < datetime.datetime(2022,4,1,0,0,0)]
print("total videos : " + str(videos_can.shape[0]))

globaltagdtf = tv810.turnserie_intodtf(nomcolonne="Total", serie_data=top_tags_global)

total videos : 70


In [9]:
for dtf in dtf_mensuel_candidat :
    mois_courant  = dtf['mois']
    dtfmois  = dtf['dataframe']
    print(mois_courant + " : " + str(dtfmois.shape[0]))

Janvier : 32
Février : 18
Mars : 20


In [10]:
# print(top_tags_global)

In [11]:
dtf = tv810.nb_tags_par_mois(listedtfmois=dtf_mensuel_candidat,toptagsglobal=top_tags_global, lst_except=lst_exceptions)
dtf = dtf.join(globaltagdtf)
dtf = dtf.sort_values(by = ["Total"], ascending = False)
dtf.head(20)

,Janvier,Février,Mars,Total
eric zemmour,32,18,20,70
zemmour président,32,18,20,70
valeurs actuelles,31,16,18,65
patriote,31,15,16,62
france,32,15,14,61
presidentielle 2022,29,15,14,58
immigration,24,12,11,47
charlotte d'ornellas,21,14,10,45
islam,25,10,8,43
actu,14,14,13,41


In [12]:
tfb = tv810.part_tags_par_mois(listedtfmois=dtf_mensuel_candidat,toptagsglobal=top_tags_global, lst_except=lst_exceptions)

In [13]:
delta_taux = tv810.gen_delta(dtfsource=tfb, colfin="Mars", coldeb="Janvier", nomcol="Delta")
delta_taux.head(20)

,Janvier,Février,Mars,Delta
les amis d'eric zemmour,50,44.4444,30,-20.000
cnews,43.75,55.5556,40,-3.750
reconquête,40.625,55.5556,45,4.375
politique,46.875,38.8889,55,8.125
face à l'info,56.25,61.1111,45,-11.250
actu,43.75,77.7778,65,21.250
islam,78.125,55.5556,40,-38.125
charlotte d'ornellas,65.625,77.7778,50,-15.625
immigration,75,66.6667,55,-20.000
presidentielle 2022,90.625,83.3333,70,-20.625
